# 4.6 Combining & Exporting Data

## Contents
1. Importing libraries and data
2. Merging dataframes
    - data consistency checks on df_ords_prior
    - combining dataframes on a common variable ('user_id')
    - merge types
3. Exporting combined dataset via pickle

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Setting path to Achievement 4 master folder
path = r'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Achievement 4- Python for DA/DEC23 Instacart Basket Analysis - MASTER FOLDER'
#checking path set correctly
path

'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Achievement 4- Python for DA/DEC23 Instacart Basket Analysis - MASTER FOLDER'

In [3]:
df_ords_prior = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'orders_products_prior.csv'),index_col = False)
df_ords =  pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_cleaned.csv'),index_col = False)

## Merging df_ords with df_ords_prior

In [4]:
# check the output

df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [6]:
df_ords_prior.shape

(32434489, 4)

In [9]:
for col in df_ords_prior.columns.tolist():
  weird = (df_ords_prior[[col]].map(type) != df_ords_prior[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords_prior[weird]) > 0:
    print (col)
  else:
    print ('No mixed columns')

No mixed columns
No mixed columns
No mixed columns
No mixed columns


In [10]:
df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [15]:
df_dups = df_ords_prior[df_ords_prior.duplicated()]

In [16]:
df_dups.head()

,order_id,product_id,add_to_cart_order,reordered


### df_ords_prior has no duplicates, mixed values, or null values. Moving on to merge with df_ords

In [17]:
df_ords.shape

(3421083, 8)

In [18]:
df_ords.head()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,orders_hour_of_day,days_since_prior_order,first_order
0,0,2539329,1,1,2,8,NaN,True
1,1,2398795,1,2,3,7,15.0,False
2,2,473747,1,3,3,12,21.0,False
3,3,2254736,1,4,4,7,29.0,False
4,4,431534,1,5,4,15,28.0,False


## TASK: Step 1

In [20]:
# Combining df_ords with df_ords_prior, on 'order_id'. Inner Merge.

df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

In [21]:
df_merged_large ['_merge'].value_counts()

_merge
both          32434489
left_only            0
right_only           0
Name: count, dtype: int64

In [26]:
df_merged_large.shape

(32434489, 11)

In [24]:
df_merged_large = df_merged_large.drop(columns = ['Unnamed: 0'])

In [25]:
df_merged_large.head()

,order_id,user_id,order_number,orders_day_of_week,orders_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,both


## Step 2

### Exporting combined orders dataset in pickle format

In [27]:
df_merged_large.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))